# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 16
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr16.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr16.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 274 274

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000161980,ENSG00000103152,ENSG00000206172,ENSG00000086506,ENSG00000076344,ENSG00000103126,ENSG00000086504,ENSG00000103202,ENSG00000242612,ENSG00000090565
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000161980,1.0000000,-0.26673003,-0.1811532,-0.22420672,0.19200856,-0.14743523,0.071889456,0.300649405,-0.42575743,-0.21750927
ENSG00000103152,-0.2667300,1.00000000,0.0965559,0.12497996,-0.07422508,-0.03522668,-0.001851756,0.007358441,0.10659253,0.24421444
ENSG00000206172,-0.1811532,0.09655590,1.0000000,0.17707732,0.11685860,0.13355319,-0.076984811,0.135401103,0.06820763,-0.09816503
ENSG00000086506,-0.2242067,0.12497996,0.1770773,1.00000000,0.06068656,-0.08464182,-0.106039274,-0.160896220,0.22834088,0.09262728
ENSG00000076344,0.1920086,-0.07422508,0.1168586,0.06068656,1.00000000,-0.25296541,-0.112545812,0.459019803,-0.50029542,0.06569731
ENSG00000103126,-0.1474352,-0.03522668,0.1335532,-0.08464182,-0.25296541,1.00000000,0.059731209,-0.282183323,0.15681113,0.16653122


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 37

[1] -0.0007858424 -0.0059427600 -0.0529740368 -0.0544117416 -0.0836656331
 [6] -0.0858330053 -0.0978440103 -0.1325870907 -0.1671437340 -0.1849334609
[11] -0.2259326974 -0.2396667654 -0.2674574237 -0.2936999353 -0.3278289115
[16] -0.3402400434 -0.3667218374 -0.3903776769 -0.4421424286 -0.4816577802
[21] -0.4971561846 -0.5222070494 -0.5423443990 -0.6208241508 -0.6289078492
[26] -0.7128617412 -0.8157570958 -0.8200940434 -0.8546855650 -0.9672269147
[31] -1.0191314463 -1.0668426299 -1.0946972505 -1.6593956806 -1.6867500940
[36] -1.8597528323 -1.9901570054

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 274 274

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000161980,ENSG00000103152,ENSG00000206172,ENSG00000086506,ENSG00000076344,ENSG00000103126,ENSG00000086504,ENSG00000103202,ENSG00000242612,ENSG00000090565
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000161980,1.0000000,-0.26673003,-0.1811532,-0.22420672,0.19200856,-0.14743523,0.071889456,0.300649405,-0.42575743,-0.21750927
ENSG00000103152,-0.2667300,1.00000000,0.0965559,0.12497996,-0.07422508,-0.03522668,-0.001851756,0.007358441,0.10659253,0.24421444
ENSG00000206172,-0.1811532,0.09655590,1.0000000,0.17707732,0.11685860,0.13355319,-0.076984811,0.135401103,0.06820763,-0.09816503
ENSG00000086506,-0.2242067,0.12497996,0.1770773,1.00000000,0.06068656,-0.08464182,-0.106039274,-0.160896220,0.22834088,0.09262728
ENSG00000076344,0.1920086,-0.07422508,0.1168586,0.06068656,1.00000000,-0.25296541,-0.112545812,0.459019803,-0.50029542,0.06569731
ENSG00000103126,-0.1474352,-0.03522668,0.1335532,-0.08464182,-0.25296541,1.00000000,0.059731209,-0.282183323,0.15681113,0.16653122


,ENSG00000161980,ENSG00000103152,ENSG00000206172,ENSG00000086506,ENSG00000076344,ENSG00000103126,ENSG00000086504,ENSG00000103202,ENSG00000242612,ENSG00000090565
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000161980,1.00000000,-0.12249968,-0.06282389,-0.095080515,0.145060417,-0.10990812,0.017779626,0.13546093,-0.14361548,-0.0791877132
ENSG00000103152,-0.12249968,1.00000000,0.02958413,0.050904295,-0.014586301,-0.01818720,-0.005890554,-0.02826560,0.02068347,0.0831899058
ENSG00000206172,-0.06282389,0.02958413,1.00000000,0.015258598,0.034530179,0.01909256,-0.014375945,0.04836529,0.04219017,0.0002010513
ENSG00000086506,-0.09508052,0.05090429,0.01525860,1.000000000,0.005962386,0.02261393,-0.089438222,-0.11273287,0.10549662,0.1039783393
ENSG00000076344,0.14506042,-0.01458630,0.03453018,0.005962386,1.000000000,-0.16223534,-0.060533284,0.24425410,-0.17724854,-0.0613565668
ENSG00000103126,-0.10990812,-0.01818720,0.01909256,0.022613931,-0.162235343,1.00000000,0.038115784,-0.14842037,0.08814465,0.0984210321


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)